In [1]:
import os
import gzip
import json
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statistics
from zipfile import ZipFile
import sys
sys.path.insert(0, "..")
from Algorithms.LExaBan.BanzhafCircuit import DNFCircuit as LExaBan
from Algorithms.LExaShap.ShapleyCircuit import DNFCircuit as LExaShap


In [2]:
lineages = []
with ZipFile(os.path.join("Example Data", 'Academic.zip'), 'r') as academic_zip:
    for file_name in academic_zip.namelist():
        with academic_zip.open(file_name) as f:
            if file_name.endswith('.json'):
                data = json.load(f)
                results = data.get("results", [])
                for result in results:
                    if len(result.get("dnf", "")) < 10000:
                        lineages.append((file_name[:-5], result.get("tuple_id", ""), result.get("dnf", "")))
print(len(lineages))


7865


In [3]:
def parse_lineage(lineage):
    """
    Parses a lineage string into a better working format.
    """
    numbered_mapping = dict()
    vars = set()
    for clause in lineage:
        vars.update(clause)
    for idx, var in enumerate(vars):
        numbered_mapping[var] = idx
    res = [set([numbered_mapping[fact] for fact in clause]) for clause in lineage]
    return res

In [ ]:
runtimes = []
for i, lineage in enumerate(lineages):
    # if i == 2341:
    if i == 2430:
        continue # skipping the lineage that takes very long to compute and only possible on strong hardware computers (see Section 5.3 in the paper)
    start = time.time()
    c = LExaBan(parse_lineage(lineage[2]))
    end = time.time()
    runtimes.append((lineage[0], lineage[1], 'LExaBan', end - start))
    try:
        start = time.time()
        c = LExaShap(parse_lineage(lineage[2]))
        end = time.time()
        runtimes.append((lineage[0], lineage[1], 'LExaShap', end - start))
    except Exception as e:
        runtimes.append((lineage[0], lineage[1], 'LExaShap', -1))
    if i % 100 == 0:
        print(f"Processed {i} lineages")

df = pd.DataFrame(runtimes, columns=["Query", "Output Tuple", "Algorithm", "Run Time"])
    

Processed 0 lineages
Processed 1 lineages
Processed 2 lineages
Processed 3 lineages
Processed 4 lineages
Processed 5 lineages
Processed 6 lineages
Processed 7 lineages
Processed 8 lineages
Processed 9 lineages
Processed 10 lineages
Processed 11 lineages
Processed 12 lineages
Processed 13 lineages
Processed 14 lineages
Processed 15 lineages
Processed 16 lineages
Processed 17 lineages
Processed 18 lineages
Processed 19 lineages
Processed 20 lineages
Processed 21 lineages
Processed 22 lineages
Processed 23 lineages
Processed 24 lineages
Processed 25 lineages
Processed 26 lineages
Processed 27 lineages
Processed 28 lineages
Processed 29 lineages
Processed 30 lineages
Processed 31 lineages
Processed 32 lineages
Processed 33 lineages
Processed 34 lineages
Processed 35 lineages
Processed 36 lineages
Processed 37 lineages
Processed 38 lineages
Processed 39 lineages
Processed 40 lineages
Processed 41 lineages
Processed 42 lineages
Processed 43 lineages
Processed 44 lineages
Processed 45 lineage

c:\Users\omera\OneDrive\Documents\GitHub\LExaBan-LExaShap\Notebooks\..\Algorithms\LExaShap\ShapleyEngine.py:175: RuntimeWarning: overflow encountered in conversion from python long
  self.grad = np.array([0,*[(factorial(i) * factorial(len(self.vars) - i - 1)) for i in range(len(self.vars))]], dtype=np.longdouble)


Processed 1194 lineages
Processed 1195 lineages
Processed 1196 lineages
Processed 1197 lineages
Processed 1198 lineages
Processed 1199 lineages
Processed 1200 lineages
Processed 1201 lineages
Processed 1202 lineages
Processed 1203 lineages
Processed 1204 lineages
Processed 1205 lineages
Processed 1206 lineages
Processed 1207 lineages
Processed 1208 lineages
Processed 1209 lineages
Processed 1210 lineages
Processed 1211 lineages
Processed 1212 lineages
Processed 1213 lineages
Processed 1214 lineages
Processed 1215 lineages
Processed 1216 lineages
Processed 1217 lineages
Processed 1218 lineages
Processed 1219 lineages
Processed 1220 lineages
Processed 1221 lineages
Processed 1222 lineages
Processed 1223 lineages
Processed 1224 lineages
Processed 1225 lineages
Processed 1226 lineages
Processed 1227 lineages
Processed 1228 lineages
Processed 1229 lineages
Processed 1230 lineages
Processed 1231 lineages
Processed 1232 lineages
Processed 1233 lineages
Processed 1234 lineages
Processed 1235 l

In [7]:
def summarize_algorithm(df):
    summary = []
    for algo in df['Algorithm'].unique():
        sub = df[df['Algorithm'] == algo]
        valid = sub[sub['Run Time'] != -1]
        total = len(sub)
        success = len(valid)
        success_rate = success / total if total > 0 else np.nan
        runtimes = valid['Run Time'].values
        mean_rt = np.mean(runtimes) if len(runtimes) > 0 else np.nan
        percentiles = np.percentile(runtimes, [50, 90, 95, 99]) if len(runtimes) > 0 else [np.nan]*4
        max_rt = np.max(runtimes) if len(runtimes) > 0 else np.nan
        summary.append({
            "Algorithm": algo,
            "Success Rate": success_rate,
            "Mean Runtime": mean_rt,
            "p50": percentiles[0],
            "p90": percentiles[1],
            "p95": percentiles[2],
            "p99": percentiles[3],
            "Max": max_rt
        })
    return pd.DataFrame(summary).set_index("Algorithm")

algo_summary = summarize_algorithm(df)
algo_summary

,Success Rate,Mean Runtime,p50,p90,p95,p99,Max
Algorithm,,,,,,,
LExaBan,1.000000,0.003568,0.000000,0.001000,0.001986,0.059640,3.905957
LExaShap,0.988428,0.010204,0.000999,0.005243,0.011533,0.046822,32.544982


In [ ]:
# Collect number of clauses and variables for each DNF
prov_stats = []
for query, tuple_id, dnf in lineages:
    clause_count = len(dnf)
    var_set = set()
    for clause in dnf:
        var_set.update(clause)
    var_count = len(var_set)
    prov_stats.append((query, tuple_id, clause_count, var_count))

prov_df = pd.DataFrame(prov_stats, columns=["Query", "Output Tuple", "Clause Count", "Variable Count"])

# Merge provenance info with runtime info
merged_df = df.merge(prov_df, on=["Query", "Output Tuple"], how="left")

# Box plot: Run Time vs Variable Count (hue=Algorithm)
plt.figure(figsize=(10, 6))
sns.boxplot(x="Variable Count", y="Run Time", hue="Algorithm", data=merged_df)
plt.yscale('log')
plt.title("Run Time vs Variable Count")
plt.tight_layout()
plt.show()

# Box plot: Run Time vs Clause Count (hue=Algorithm)
plt.figure(figsize=(10, 6))
sns.boxplot(x="Clause Count", y="Run Time", hue="Algorithm", data=merged_df)
plt.yscale('log')
plt.title("Run Time vs Clause Count")
plt.tight_layout()
plt.show()

# Bar plot: Success Rate vs Variable Count
success_var = merged_df.copy()
success_var["Success"] = success_var["Run Time"] != -1
success_rate_var = success_var.groupby("Variable Count")["Success"].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x="Variable Count", y="Success", data=success_rate_var)
plt.title("Success Rate vs Variable Count")
plt.ylabel("Success Rate")
plt.tight_layout()
plt.show()

# Bar plot: Success Rate vs Clause Count
success_clause = merged_df.copy()
success_clause["Success"] = success_clause["Run Time"] != -1
success_rate_clause = success_clause.groupby("Clause Count")["Success"].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x="Clause Count", y="Success", data=success_rate_clause)
plt.title("Success Rate vs Clause Count")
plt.ylabel("Success Rate")
plt.tight_layout()
plt.show()